In [2]:
import os

from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from random import randint

In [3]:
# every frame is considered independent

no_of_features = 13
no_of_fbank_features = 20
no_of_columns = (3 * no_of_features) + no_of_fbank_features

def get_feature_vectors(dataset_type):
    
    #set parameters for training and testing
    if (dataset_type == "train"):
        directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/enroll')
        no_of_frames = 400
        start_frame = 1
    elif (dataset_type == "test"):    
        directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/test')
        no_of_frames = 40
        start_frame = randint(1, 200)
        
    dataset = numpy.empty([0, no_of_columns + 1])
    
    for file in os.listdir(directory):
        
        # filter speakers
        names = ['F101', 'F102', 'F103', 'F104', 'F105', 'M101', 'M102', 'M103', 'M104']

        if any(name in file for name in names):
            
            # extract mfcc vectors
            (rate,sig) = wav.read(os.path.join(directory, file))
            fbank_feat = logfbank(sig,rate)
            mfcc_feat = mfcc(sig,rate)
            d_mfcc_feat = delta(mfcc_feat, 2)
            dd_mfcc_feat = delta(d_mfcc_feat, 2)
            
#             fbank_feat = logfbank(sig,rate)
            mfcc_vectors = mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            dmfcc_vectors = d_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            ddmfcc_vectors = dd_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            fbank_vectors = fbank_feat[start_frame:start_frame+no_of_frames,:no_of_fbank_features]
            
            feature_vectors = numpy.hstack((mfcc_vectors, dmfcc_vectors, ddmfcc_vectors, fbank_vectors))
#             print(feature_vectors.shape)
            
            # get speaker index from filename
            speaker_index = file.split("_")[0]
            if speaker_index[0] == 'M':
                speaker_index = 5 + int(speaker_index[3:])
            else:
                speaker_index = int(speaker_index[3:])

            #append speaker index to feature vectors
            np_speaker_index = numpy.array([speaker_index])
            temp = numpy.tile(np_speaker_index[numpy.newaxis,:], (feature_vectors.shape[0],1))
            concatenated_feature_vector = numpy.concatenate((feature_vectors,temp), axis=1)
            
#             print(concatenated_feature_vector.shape)
#             print(fbank_vectors.shape)
            
            # append file's data to dataset
            dataset = numpy.concatenate((dataset, concatenated_feature_vector), axis=0)
            

    return dataset


In [4]:
from keras.models import Sequential
import numpy as numpy 

Using Theano backend.


In [5]:
# from numpy import genfromtxt
my_data = get_feature_vectors("train")


In [6]:
# print(my_data)
print(my_data.shape)

(3600, 60)


In [7]:
Y = numpy.copy(my_data[:, no_of_columns:])
print(Y.shape)

(3600, 1)


In [8]:
X = numpy.copy(my_data[:, :no_of_columns])
print(X.shape)
mean = X.mean(0, keepdims=True)

print(mean.shape)
std_deviation = numpy.std(X, axis=0, keepdims=True)
print(std_deviation.shape)

normalized_X = (X - mean) / std_deviation
print(normalized_X.shape)


(3600, 59)
(1, 59)
(1, 59)
(3600, 59)


In [18]:
from keras import utils as np_utils
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

one_hot_labels = np_utils.to_categorical(Y, num_classes=10)
print(one_hot_labels)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [44]:
def train(layers):

    # MultiLayer Perceptron
    model = Sequential()

    # 3000 0.6 1500 0.6 -> 57% acc
    # 2560 0.7 1280 0.7 -> 60% acc
    # 2560 0.7 1600 0.7 -> 51% acc
    # 2560 0.6 1600 0.7 -> 49%
    # 2560 0.7 1280 -   -> 50
    # 3000 0.7 1280 0.7 -> 54% acc
    # 2560 0.7 1280 0.8 -> 45% acc

    model.add(Dense(2560, activation='tanh', input_dim=no_of_columns))
    model.add(Dropout(0.7))
    model.add(Dense(1280, activation='tanh'))
    model.add(Dropout(0.7))
    
    for layer in layers:
        model.add(Dense(layer, activation='tanh'))
    
    model.add(Dense(10, activation='softmax'))

    sgd = SGD(lr=0.005, decay=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    model.fit(normalized_X, one_hot_labels, epochs=10, batch_size=32)
    
    return model
# score = model.evaluate(x_test, y_test, batch_size=128)

In [53]:
def test(model):
    
    test_model = get_feature_vectors("test")
#     print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=10)

#     print(model.test_on_batch(test_X, test_labels, sample_weight=None))
#     print(model.metrics_names)
    predictions = model.predict(test_X)
    
    b = [sum(predictions[current: current+40]) for current in range(0, len(predictions), 40)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))
        
    diff = predicted_Y - test_Y[::40].T[0]
    correct = sum(x == 0 for x in diff)
    total = len(predicted_Y)
    percent = correct/total * 100
    print(str(correct) + " / " + str(total) + " = " + str(percent))
    
    loss = str(numpy.sum(numpy.abs(diff)))
    print("Loss = " + loss)
    
    return correct, percent, loss

In [56]:
# with open('automate_op.csv', 'a') as the_file:
#     the_file.write('Hello\n')

# print("Test", file=f)
for i in range(100, 2001,300):  
    for j in range(100,2001,300):  
        print(str(i) + ", " + str(j))
        model = train([i, j])
        
        correct, percent, loss = test(model)
        string = "2560, 0.7, 1280, 0.7, " + str(i) + ", " + str(j) + ", " + str(correct) + ", " + str(percent) + ", " + str(loss)
        f = open('automate_op.csv', 'a')
        print(string, file=f)
        f.close()
        
        correct, percent, loss = test(model)
        string = "2560, 0.7, 1280, 0.7, " + str(i) + ", " + str(j) + ", " + str(correct) + ", " + str(percent) + ", " + str(loss)
        f = open('automate_op.csv', 'a')
        print(string, file=f)
        f.close()
        
        correct, percent, loss = test(model)
        string = "2560, 0.7, 1280, 0.7, " + str(i) + ", " + str(j) + ", " + str(correct) + ", " + str(percent) + ", " + str(loss)
        f = open('automate_op.csv', 'a')
        print(string, file=f)
        f.close()

100, 100
Epoch 1/10
3600/3600 [==============================] - 9s 3ms/step - loss: 1.9555 - acc: 0.3061
Epoch 2/10
3600/3600 [==============================] - 9s 3ms/step - loss: 1.5039 - acc: 0.4675
Epoch 3/10
3600/3600 [==============================] - 9s 3ms/step - loss: 1.3787 - acc: 0.5092
Epoch 4/10
3600/3600 [==============================] - 10s 3ms/step - loss: 1.2506 - acc: 0.5503
Epoch 5/10
3600/3600 [==============================] - 9s 3ms/step - loss: 1.1820 - acc: 0.5761
Epoch 6/10
3600/3600 [==============================] - 9s 3ms/step - loss: 1.0972 - acc: 0.6103
Epoch 7/10
3600/3600 [==============================] - 10s 3ms/step - loss: 1.0117 - acc: 0.6422
Epoch 8/10
3600/3600 [==============================] - 9s 3ms/step - loss: 0.9718 - acc: 0.6506
Epoch 9/10
3600/3600 [==============================] - 10s 3ms/step - loss: 0.9058 - acc: 0.6775
Epoch 10/10
3600/3600 [==============================] - 10s 3ms/step - loss: 0.8445 - acc: 0.7031
53 / 90 = 58.888

Epoch 6/10
3600/3600 [==============================] - 10s 3ms/step - loss: 1.0706 - acc: 0.6122
Epoch 7/10
3600/3600 [==============================] - 10s 3ms/step - loss: 0.9885 - acc: 0.6486
Epoch 8/10
3600/3600 [==============================] - 10s 3ms/step - loss: 0.9347 - acc: 0.6664
Epoch 9/10
3600/3600 [==============================] - 10s 3ms/step - loss: 0.8781 - acc: 0.6833
Epoch 10/10
3600/3600 [==============================] - 10s 3ms/step - loss: 0.8293 - acc: 0.7047
50 / 90 = 55.5555555556
Loss = 117.0
52 / 90 = 57.7777777778
Loss = 118.0
56 / 90 = 62.2222222222
Loss = 111.0
400, 400
Epoch 1/10
3600/3600 [==============================] - 10s 3ms/step - loss: 1.9186 - acc: 0.3211
Epoch 2/10
3600/3600 [==============================] - 10s 3ms/step - loss: 1.5179 - acc: 0.4714
Epoch 3/10
3600/3600 [==============================] - 10s 3ms/step - loss: 1.3788 - acc: 0.5131
Epoch 4/10
3600/3600 [==============================] - 10s 3ms/step - loss: 1.3164 - acc: 0.54

3600/3600 [==============================] - 10s 3ms/step - loss: 0.7949 - acc: 0.7144
53 / 90 = 58.8888888889
Loss = 101.0
52 / 90 = 57.7777777778
Loss = 113.0
49 / 90 = 54.4444444444
Loss = 125.0
700, 400
Epoch 1/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.9096 - acc: 0.3353
Epoch 2/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.5170 - acc: 0.4731
Epoch 3/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.4072 - acc: 0.5094
Epoch 4/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.3211 - acc: 0.5336: 0s - loss: 1.3203 - acc: 0
Epoch 5/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.2356 - acc: 0.5667
Epoch 6/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.1580 - acc: 0.5844
Epoch 7/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.0938 - acc: 0.6156
Epoch 8/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.

3600/3600 [==============================] - 11s 3ms/step - loss: 1.4091 - acc: 0.5181
Epoch 4/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.3184 - acc: 0.5333
Epoch 5/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.2162 - acc: 0.5669
Epoch 6/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.1690 - acc: 0.5831
Epoch 7/10
3600/3600 [==============================] - 11s 3ms/step - loss: 1.0908 - acc: 0.6081
Epoch 8/10
3600/3600 [==============================] - 11s 3ms/step - loss: 0.9987 - acc: 0.6397
Epoch 9/10
3600/3600 [==============================] - 12s 3ms/step - loss: 0.9476 - acc: 0.6572
Epoch 10/10
3600/3600 [==============================] - 11s 3ms/step - loss: 0.8875 - acc: 0.6844
42 / 90 = 46.6666666667
Loss = 153.0
48 / 90 = 53.3333333333
Loss = 131.0
48 / 90 = 53.3333333333
Loss = 117.0
1000, 700
Epoch 1/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.9325 - acc: 0.3228
Epoch 2

3600/3600 [==============================] - 12s 3ms/step - loss: 0.9869 - acc: 0.6469
Epoch 9/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.9320 - acc: 0.6675
Epoch 10/10
3600/3600 [==============================] - 15s 4ms/step - loss: 0.8758 - acc: 0.6839
52 / 90 = 57.7777777778
Loss = 115.0
57 / 90 = 63.3333333333
Loss = 94.0
48 / 90 = 53.3333333333
Loss = 117.0
1300, 700
Epoch 1/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.9254 - acc: 0.3261
Epoch 2/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.5915 - acc: 0.4547
Epoch 3/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.4549 - acc: 0.4908
Epoch 4/10
3600/3600 [==============================] - 13s 3ms/step - loss: 1.3448 - acc: 0.5333
Epoch 5/10
3600/3600 [==============================] - 12s 3ms/step - loss: 1.2932 - acc: 0.5519
Epoch 6/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.2108 - acc: 0.5692
Epoch 7/

3600/3600 [==============================] - 14s 4ms/step - loss: 1.9515 - acc: 0.3164
Epoch 2/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.5999 - acc: 0.4519
Epoch 3/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.4446 - acc: 0.4989
Epoch 4/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.3597 - acc: 0.5181
Epoch 5/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.2823 - acc: 0.5558
Epoch 6/10
3600/3600 [==============================] - 15s 4ms/step - loss: 1.1866 - acc: 0.5872
Epoch 7/10
3600/3600 [==============================] - 14s 4ms/step - loss: 1.1342 - acc: 0.6019
Epoch 8/10
3600/3600 [==============================] - 13s 4ms/step - loss: 1.0442 - acc: 0.6228
Epoch 9/10
3600/3600 [==============================] - 14s 4ms/step - loss: 0.9878 - acc: 0.6417
Epoch 10/10
3600/3600 [==============================] - 13s 4ms/step - loss: 0.9523 - acc: 0.6564
50 / 90 = 55.5555555556
Loss =

3600/3600 [==============================] - 16s 4ms/step - loss: 1.2173 - acc: 0.5639
Epoch 7/10
3600/3600 [==============================] - 16s 5ms/step - loss: 1.1073 - acc: 0.6067
Epoch 8/10
3600/3600 [==============================] - 16s 4ms/step - loss: 1.0743 - acc: 0.6164
Epoch 9/10
3600/3600 [==============================] - 15s 4ms/step - loss: 0.9946 - acc: 0.6442
Epoch 10/10
3600/3600 [==============================] - 15s 4ms/step - loss: 0.9259 - acc: 0.6731
46 / 90 = 51.1111111111
Loss = 130.0
52 / 90 = 57.7777777778
Loss = 123.0
51 / 90 = 56.6666666667
Loss = 118.0
1900, 1000
Epoch 1/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.9312 - acc: 0.3328
Epoch 2/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.6264 - acc: 0.4439
Epoch 3/10
3600/3600 [==============================] - 17s 5ms/step - loss: 1.4959 - acc: 0.4819
Epoch 4/10
3600/3600 [==============================] - 16s 5ms/step - loss: 1.4000 - acc: 0.5103
Epoch 

In [55]:

    
# print(predicted_Y)
# print(test_Y[::40].T)

# for t, p in zip(test_Y[::40].T[0], predicted_Y):
#     print (int(t), p)

f.close()

In [28]:
# majority

# argmax_pred = numpy.argmax(predictions, axis=1)
# argmax_pred = argmax_pred.reshape((-1, 25))

# from scipy.stats import mode
# argmax_pred = mode(argmax_pred, axis=-1)[0]

# # print(argmax_pred)
# # print(test_Y)

# for t, p in zip(test_Y[::40].T[0], argmax_pred):
#     print (int(t), p)

In [17]:
# def find_majority(k):
#     myMap = {}
#     maximum = ( '', 0 ) # (occurring element, occurrences)
#     for n in k:
#         if n in myMap: myMap[n] += 1
#         else: myMap[n] = 1

#         # Keep track of maximum on the go
#         if myMap[n] > maximum[1]: maximum = (n,myMap[n])

#     return maximum